## 1. Import

In [ ]:
import requests # To read html text
from bs4 import BeautifulSoup # To parse html text
import re # Regular expressions
import pandas as pd
# You can also import other thing ...

## 2. Danh sách url

In [ ]:
urls = ['https://tiki.vn/tieu-thuyet-tinh-cam-lang-man/c844?src=tree',
    'https://tiki.vn/sach-van-hoc-viet-nam/c2547?src=tree',
    'https://tiki.vn/van-hoc-nuoc-ngoai/c2546?src=tree',
    'https://tiki.vn/tac-pham-kinh-dien/c842?src=tree',
    'https://tiki.vn/truyen-ngan/c845?src=tree',
    'https://tiki.vn/trinh-tham/c664?src=tree',
    'https://tiki.vn/truyen-kinh-di/c665?src=tree',
    'https://tiki.vn/huyen-bi-gia-tuong/c840?src=tree',
    'https://tiki.vn/tieu-su-hoi-ky/c843?src=tree',
    'https://tiki.vn/kiem-hiep-vo-hiep/c639?src=tree',
    'https://tiki.vn/du-ky/c4933?src=tree',
    'https://tiki.vn/phong-su-ky-su/c5245?src=tree',
    'https://tiki.vn/phe-binh-van-hoc/c5244?src=tree',
    'https://tiki.vn/truyen-dai/c6750?src=tree',
    'https://tiki.vn/truyen-hai-huoc-truyen-cuoi/c444?src=tree',
    'https://tiki.vn/tho/c446?src=tree',
    'https://tiki.vn/light-novel/c7358?src=tree',
    'https://tiki.vn/truyen-tranh/c8973?src=tree',
    'https://tiki.vn/mau-giao-0-6-tuoi/c5936?src=tree',
    'https://tiki.vn/nhi-dong-6-11-tuoi/c3369?src=tree',
    'https://tiki.vn/thieu-nien-11-15-tuoi/c3370?src=tree',
    'https://tiki.vn/van-hoc-thieu-nhi/c1754?src=tree',
    'https://tiki.vn/kien-thuc-ky-nang/c853?src=tree',
    'https://tiki.vn/bai-hoc-dao-duc/c852?src=tree',
    'https://tiki.vn/truyen-co-tich/c1753?src=tree',
    'https://tiki.vn/truyen-ke-cho-be/c854?src=tree',
    'https://tiki.vn/truyen-tranh-thieu-nhi/c855?src=tree',
    'https://tiki.vn/to-mau-luyen-chu/c856?src=tree',
    'https://tiki.vn/sach-ky-nang-song/c871?src=tree',
    'https://tiki.vn/sach-nghe-thuat-song-dep/c872?src=tree',
    'https://tiki.vn/sach-huong-nghiep-ky-nang/c716?src=tree',
    'https://tiki.vn/sach-ba-me-em-be/c2527?src=tree',
    'https://tiki.vn/sach-kinh-te/c846?src=tree',
    'https://tiki.vn/sach-tham-khao/c2320?src=tree',
    'https://tiki.vn/sach-giao-khoa-cac-cap/c2321?src=tree',
    'https://tiki.vn/tu-dien/c897?src=tree',
    'https://tiki.vn/ngoai-ngu-tu-dien/c887?src=tree',
    'https://tiki.vn/truyen-tranh/c1084?src=tree',
    'https://tiki.vn/gia-trinh-dai-hoc-cao-dang/c6991?src=tree',
    'https://tiki.vn/sach-kien-thuc-tong-hop/c873?src=tree',
    'https://tiki.vn/education-teaching/c5308?src=tree',
    'https://tiki.vn/fiction-literature/c9?src=tree',
    'https://tiki.vn/self-help/c614?src=tree',
    'https://tiki.vn/children-books/c7?src=tree',
    'https://tiki.vn/culture-and-art/c623?src=tree',
    'https://tiki.vn/teens/c31?src=tree',
    'https://tiki.vn/economics/c4?src=tree',
    'https://tiki.vn/awards/c616?src=tree',
    'https://tiki.vn/social-sciences/c27?src=tree',
    'https://tiki.vn/magazines/c6445?src=tree']

## 3. Chương trình chính

In [ ]:
# Read html text and parse it
li_books = pd.DataFrame()
idx = 0
for url in urls:
    
    print 'url is: ', url, 'running...'
    
    html_text = requests.get(url).text
    tree = BeautifulSoup(html_text, 'lxml')

    # topic
    topic = tree.find_all(['h1'])[-1].string[25:-22]
    
    # num_books
    is_num_books = tree.find_all(['h4'])[5].string
    
    try:
        num_books = is_num_books.split()[0]
    except:
        num_books = tree.find_all(['h4'])[6].string.split()[0]

    num_books = int(num_books)

    # num_pages
    num_pages = (num_books - 1) / 24 + 1 # 24 books each page

    print 'Topic: ', topic
    print 'Num of books: ', num_books
    print 'Num of pages: ', num_pages
    print '-' * 50

    for page_idx in range(num_pages):

        _url = url + '&page=' + str(page_idx + 1)
        html_text = requests.get(_url).text
        tree = BeautifulSoup(html_text, 'lxml')

        books = tree.find_all(class_='product-item')        

        for book in books:
            # data id
            data_id = book.find('a')['data-id']

            # link book data
            link = book.find('a')['href']
            try:
                html_text = requests.get(link).text
            except:
                continue

            tree = BeautifulSoup(html_text, 'lxml')

            # title
            title = tree.find_all('title')[-1].get_text().split('|')[0][:-1]

            # rating and rating_Count
            about_rate = tree.find_all('meta')[-2:]

            try:
                rating = float(about_rate[0]['content'])
            except:
                rating = 0

            rating = rating * (rating <= 5.0)

            try:
                rating_Count = int(about_rate[1]['content'])
            except:
                rating_Count = 0

            # nxb, author and publidation date
            information = tree.find_all('td')[4:]

            _nxb1 = u'Nh\xe0 xu\u1ea5t b\u1ea3n'
            _nxb2 = u'Manufacturer'
            _author = u'T\xe1c gi\u1ea3'
            _date = u'Ng\xe0y xu\u1ea5t b\u1ea3n'

            nxb = 'Unidentify'
            author = 'Unidentify'
            date = 'Unidentify'

            for i in range(0, len(information), 2):
                text = information[i].get_text()
                if (_nxb1 in text) or (_nxb2 in text):
                    nxb = information[i + 1].get_text()[85:-117]
                if _author in text:
                    author = information[i + 1].get_text()[85:-117]
                if _date in text:
                    date = information[i + 1].get_text()[85:-117]

            # review
            review = book.find(class_='review')
            review = review.get_text()
            review = review.split(" ")[-1][1:]
            try:
                review = int(review)
            except:
                review = 0

            # price sale
            prices = book.find(class_='price-sale')
            for ele in prices.find_all(['span']):
                ele.extract()
            price = prices.get_text().replace('.', '')
            price = int(price.replace(' ', '')[:-4])
            
            li_books = li_books.append(pd.DataFrame(data = {'id': data_id.encode('utf-8'), 'title': title.encode('utf-8'),
                                                            'topic': topic.encode('utf-8'), 'nxb': nxb.encode('utf-8'),
                                                            'author': author.encode('utf-8'), 'date': date.encode('utf-8'),
                                                            'review': review, 'rating': rating, 'rating_Count': rating_Count,
                                                            'price': price}, index=[idx]))
            idx += 1

## 4. Chuẩn hóa dữ liệu

Đặt tên cho cột index (name = 'idx')

In [ ]:
li_books.index.name = 'idx'

Điền các giá trị còn thiếu trong cột 'author' bằng giá trị 'Unidentify'

In [ ]:
missing_author_mask = data1.author.isnull()
for idx in li_books.author[missing_author_mask].index:
    li_books.author.set_value(idx, 'Unidentify')

Chuẩn hóa lại các giá trị trong cột price

In [ ]:
for i in range(len(li_books)):
    if li_books.price[i] < 84:
        li_books.price.set_value(i, li_books.price[i]*1000)
    elif li_books.price[i] < 8400:
        li_books.price.set_value(i, li_books.price[i]*100)

Xáo trộn các mẫu dữ liệu (thực ra là sắp xếp theo cột 'id')

In [ ]:
li_books = li_books.sort_values(by='id')

Phân các mẫu dữ liệu vào lần lượt các tập training set (20000 mẫu), test set (5000 mẫu), validation set (số mẫu còn lại)

In [ ]:
train = li_books[:20000]
test = li_books[20000:25000]
val = li_books[25000:]

Ghi dữ liệu vào file

In [ ]:
train.to_csv('train.csv')
test.to_csv('test.csv')
val.to_csv('val.csv')